<a href="https://colab.research.google.com/github/batiukmaks/NLP-Labs/blob/main/labs/LPNLP7_RNN_Quest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN Quest

In [1]:
!pip install --quiet --ignore-installed http://nlp.band/static/pypy/lpnlp-2023.10.2-py3-none-any.whl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 6.4 MB/s eta 0:00:00


In [3]:
!pwd

/content


In [4]:
import lpnlp

lab = lpnlp.start(
    email="maksym.batiuk.kn.2021@lpnu.ua",  # <---- Заповніть це поле
    lab="quest_rnn"
)


Ваше завдання: http://nlp.band/static/quest-rnn/34452750d6b0.zip. Удачі! █


1. Завантажте пакет із завданням за посиланням вище ^
2. Прочитайте README
3. Виконайте завдання
  - Завдання не залежить від мови програмування
  - Завдання не залежить від фреймоворку. Pytorch, NumPy, Keras або взагалі без сторонніх бібліотек -- все можливо
4. Відправте відповідь нижче

In [5]:
!wget http://nlp.band/static/quest-rnn/34452750d6b0.zip

--2024-11-02 09:25:31--  http://nlp.band/static/quest-rnn/34452750d6b0.zip
Resolving nlp.band (nlp.band)... 51.12.243.211
Connecting to nlp.band (nlp.band)|51.12.243.211|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 518811 (507K) [application/zip]
Saving to: ‘34452750d6b0.zip’

34452750d6b0.zip    100%[===================>] 506.65K  1008KB/s    in 0.5s    

2024-11-02 09:25:32 (1008 KB/s) - ‘34452750d6b0.zip’ saved [518811/518811]



In [6]:
!unzip 34452750d6b0.zip

Archive:  34452750d6b0.zip
 extracting: embedding.weight.json   
 extracting: W_h.weight.json         
 extracting: W_h.bias.json           
 extracting: U_h.weight.json         
 extracting: U_h.bias.json           
 extracting: W_y.weight.json         
 extracting: vocab.json              
 extracting: quest.bin               
 extracting: README                  


In [79]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import json

In [86]:
class ElmanRNN(nn.Module):
    def __init__(self, embedding_dim=48, hidden_dim=160, vocab_size=133):
        super(ElmanRNN, self).__init__()

        # Initialize dimensions
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size

        # Define layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.W_h = nn.Linear(embedding_dim, hidden_dim)
        self.U_h = nn.Linear(hidden_dim, hidden_dim)
        self.W_y = nn.Linear(hidden_dim, vocab_size)

    def forward(self, message):
        # Initialize hidden state (h_0)
        h_t = torch.zeros(message.size(0), self.hidden_dim, device=message.device)
        outputs = []

        for t in range(message.size(1)):
            # Embedding lookup for x_t
            if t == 0:
                x_t = self.embedding(message[:, t])  # Use the input at time step 0
            else:
                x_t = self.embedding(message_t)  # Use predicted token from previous step

            # Update hidden state using Elman RNN equations
            h_t = torch.tanh(self.W_h(x_t) + self.U_h(h_t))

            # Compute output and save the prediction
            y_t = self.W_y(h_t)
            outputs.append(y_t.unsqueeze(1))

            # Vocabulary lookup for next input token
            message_t = torch.argmax(y_t, dim=1)

        # Concatenate outputs for the sequence
        outputs = torch.cat(outputs, dim=1)
        return outputs

    def load_weights(self, weight_paths):
        """
        Load weights from JSON files and assign them to model parameters.

        Args:
        weight_paths (dict): A dictionary containing file paths for each weight file.
                             Expected keys: 'U_h_weight', 'U_h_bias', 'W_h_weight',
                             'W_h_bias', 'W_y_weight', 'embedding_weight'.
        """
        def load_weight_from_json(file_path):
            with open(file_path, 'r') as f:
                weight_data = json.load(f)
            return torch.tensor(weight_data)

        # Assign weights to the model's parameters
        self.U_h.weight.data = load_weight_from_json(weight_paths['U_h_weight'])
        self.U_h.bias.data = load_weight_from_json(weight_paths['U_h_bias'])
        self.W_h.weight.data = load_weight_from_json(weight_paths['W_h_weight'])
        self.W_h.bias.data = load_weight_from_json(weight_paths['W_h_bias'])
        self.W_y.weight.data = load_weight_from_json(weight_paths['W_y_weight'])
        self.embedding.weight.data = load_weight_from_json(weight_paths['embedding_weight'])

        print("Weights loaded successfully.")

    def forward_pass(self, start_index, end_index, max_sequence=100):
        """
        Generate a sequence starting from `start_index` until reaching `end_index` or
        until `max_sequence` length is reached.

        Args:
        start_index (int): The index of the starting token.
        end_index (int): The index of the end token.
        max_sequence (int): The maximum length of the generated sequence to prevent infinite loops.

        Returns:
        list: A list of token indices representing the generated sequence.
        """
        sequence = [start_index]
        h_t = torch.zeros(1, self.hidden_dim, device=self.embedding.weight.device)

        current_index = start_index
        for _ in range(max_sequence):
            # Get the embedding of the current token
            x_t = self.embedding(torch.tensor([current_index], device=self.embedding.weight.device))

            # Update hidden state
            h_t = torch.tanh(self.W_h(x_t) + self.U_h(h_t))

            # Get the output and determine the next token
            y_t = self.W_y(h_t)

            current_index = torch.argmax(y_t, dim=-1).item()
            sequence.append(current_index)

            # Break if the end token is reached
            if current_index == end_index:
                break

        return sequence

# Example usage
weight_paths = {
    'U_h_weight': 'U_h.weight.json',
    'U_h_bias': 'U_h.bias.json',
    'W_h_weight': 'W_h.weight.json',
    'W_h_bias': 'W_h.bias.json',
    'W_y_weight': 'W_y.weight.json',
    'embedding_weight': 'embedding.weight.json'
}

# Model instantiation
embedding_dim = 48
hidden_dim = 160
vocab_size = 133
model = ElmanRNN(embedding_dim, hidden_dim, vocab_size)
model.load_weights(weight_paths)

Weights loaded successfully.


In [81]:
class Vocabulary:
    def __init__(self, tokens):
        self.tokens = tokens
        self._itos = list(tokens)
        self._stoi = {token: index for index, token in enumerate(self._itos)}

    def stoi(self, token: str) -> int:
        """Return token index or `unk_index` if `token` is not in the vocab."""
        return self._stoi.get(token, None)

    def itos(self, index: int) -> str:
        """Return token by its `index`."""
        return self._itos[index]

    def __len__(self) -> int:
        return len(self._itos)

def vectorize(tokens, vocab: Vocabulary):
    """Convert a list of tokens to a tensor of indices."""
    return torch.tensor([vocab.stoi(token) for token in tokens], dtype=torch.long)

# Load the vocabulary from a JSON file and initialize Vocabulary
with open('vocab.json', 'r') as f:
    tokens = json.load(f)

vocab = Vocabulary(tokens=tokens)

In [85]:
start_index = vocab.stoi("[")
end_index = vocab.stoi("]")

# Generate sequence using forward_pass
generated_sequence_indices = model.forward_pass(start_index=start_index, end_index=end_index)
generated_sequence_tokens = [vocab.itos(idx) for idx in generated_sequence_indices]

print("Generated sequence:", "".join(generated_sequence_tokens))


Generated sequence: [Назва треку: https://www.youtube.com/watch?v=WTwPWIGMkvs]


In [78]:
lab.answer("Це і є життя")

Відповідь правильна ✅
You did it! 🚀 Next step: https://tally.so/r/wMXM0p
